In [1]:
import math

## Diffie-Hellman

In [2]:
def DH_public_key(g, a, p):
    return pow(g, a, p)

In [3]:
def DH_shared_secret(public_key, a, p):
    return pow(public_key, a, p)

## Modulo arithmetics

In [4]:
def inverse(x, p):
    return pow(x, -1, p)

In [5]:
# Chinese Remainder Theorem
# x = r[i] % n[i]

def CRT(rs, ns):

    N = math.prod(ns)
    x = 0

    for i in range(len(ns)):
        M = int(N / ns[i])
        v = inverse(M, ns[i])
        e = v * M
        x += rs[i] * e

    return x % N

In [6]:
def order(a, n):

    if not math.gcd(a, n) == 1:
        return None

    else:
        product = 1
        k = 1

        while k < n:
            product = (product * a) % n
            if product == 1:
                return k
            k += 1
        return None

## Circle: $ x^{2} + y^{2} = 1 $

In [7]:
def circle_addition(a, b, p):
    x = (a[0] * b[1] + a[1] * b[0]) % p
    y = (a[1] * b[1] - a[0] * b[0]) % p
    return x, y

In [8]:
def DLP_circle(P, P_A, p):
    result = P
    a = 1
    stop = False
    while not stop:
        result = circle_addition(result, P, p)
        if result == P_A:
            stop = True
        a += 1
    return a

## Twisted Edwards curves: $ ax^{2} + y^{2} = 1 + dx^{2}y^{2} $
## Montgomery curves: $ Bv^{2} = u^{3} + Au^{2} + u $

In [9]:
def twisted_to_montgomery(a, d, p, print_intermediate=False):

    A = (2 * (a + d) * inverse(a - d, p)) % p
    B = (4 * inverse(a - d, p)) % p

    if print_intermediate:
        print('A = ' + str(A))
        print('B = ' + str(B))

    return A, B

In [10]:
def twisted_to_montgomery_point(P, p, print_intermediate=False):

    x, y = P[0], P[1]
    u = ((1 + y) * inverse(1 - y, p)) % p
    v = (u * inverse(x, p)) % p

    if print_intermediate:
        print('P_m = ' + str((u, v)))

    return u, v

In [11]:
def montgomery_to_twisted(A, B, p, print_intermediate=False):

    a = ((A + 2) * inverse(B, p)) % p
    d = ((A - 2) * inverse(B, p)) % p

    if print_intermediate:
        print('a = ' + str(a))
        print('d = ' + str(d))

    return a, d

In [12]:
def montgomery_to_twisted_point(P, p, print_intermediate=False):

    u, v = P[0], P[1]
    x = (u * inverse(v, p)) % p
    y = ((u - 1) * inverse(u + 1, p)) % p

    if print_intermediate:
        print('P_t = ' + str((x, y)))

    return x, y

In [13]:
def twisted_addition(P, Q, a, d, p):

    x_1, y_1 = P[0], P[1]
    x_2, y_2 = Q[0], Q[1]

    x = ((x_1 * y_2 + y_1 * x_2) * inverse(1 + d * x_1 * x_2 * y_1 * y_2, p)) % p
    y = ((y_1 * y_2 - a * x_1 * x_2) * inverse(1 - d * x_1 * x_2 * y_1 * y_2, p)) % p

    return x, y

In [14]:
def twisted_multiple_additions(P, k, a, d, p, print_intermediate=False):

    Q = P
    if print_intermediate:
        print('1P = ' + str(Q))

    for i in range(k - 1):
        Q = twisted_addition(Q, P, a, d, p)

        if print_intermediate:
            print(str(i + 2) + 'P = ' + str(Q))

    return Q

In [15]:
def is_on_twisted_curve(P, a, d, p, print_intermediate=False):

    x, y = P[0], P[1]

    left_side = (a * x ** 2 + y ** 2) % p
    right_side = (1 + d * x ** 2 * y ** 2) % p

    if print_intermediate:
        print(left_side, right_side)

    return left_side == right_side

## Weierstrass curves
## General form: $ y^{2} + c_{1}xy + c_{3}y = x^{3} + c_{2}x^{2} + c_{4}x + c_{6} $
## Short form: $ y^{2} = x^{3} + c_{4}x + c_{6} $

In [16]:
def general_to_short(cs, p):

    c_1, c_2, c_3, c_4, c_6 = cs[0], cs[1], cs[2], cs[3], cs[4]

    short_c_4 = ((-1 * inverse(48, p) * c_1 ** 4) + (-1 * inverse(6, p) * c_2 * c_1 ** 2) + (inverse(2, p) * c_3 * c_1) + (-1 * inverse(3, p) * c_2 ** 2) + c_4) % p
    short_c_6 = ((inverse(864, p) * c_1 ** 6) + (inverse(72, p) * c_2 * c_1 ** 4) + (-1 * inverse(24, p) * c_3 * c_1 ** 3) + (inverse(18, p) * c_2 ** 2 * c_1 ** 2) +
                 (-1 * inverse(12, p) * c_4 * c_1 ** 2) + (-1 * inverse(6, p) * c_2 * c_3 * c_1) + (2 * inverse(27, p) * c_2 ** 3) + (inverse(4, p) * c_3 ** 2) +
                 (-1 * inverse(3, p) * c_2 * c_4) + c_6) % p

    return [0, 0, 0, short_c_4, short_c_6]

In [17]:
def general_to_short_point(P, cs, p):

    if P is None:
        return P

    c_1, c_2, c_3, c_4, c_6 = cs[0], cs[1], cs[2], cs[3], cs[4]

    y = (P[1] + inverse(2, p) * (c_1 * P[0] + c_3)) % p
    x = (P[0] + inverse(3, p) * (c_2 + inverse(4, p) * c_1 ** 2)) % p

    return x, y

In [18]:
def short_to_general_point(P, cs, p):

    if P is None:
        return P

    c_1, c_2, c_3, c_4, c_6 = cs[0], cs[1], cs[2], cs[3], cs[4]

    y = (P[1] - inverse(2, p) * (c_1 * P[0] + c_3)) % p
    x = (P[0] - inverse(3, p) * (c_2 + inverse(4, p) * c_1 ** 2)) % p

    return x, y

In [19]:
def is_on_weierstrass_curve(P, cs, p):
    x = P[0]
    y = P[1]
    left_side = (y ** 2 + cs[0] * x * y + cs[2] * y) % p
    right_side = (x ** 3 + cs[1] * x ** 2 + cs[3] * x + cs[4]) % p
    return left_side == right_side

In [20]:
def weierstrass_number_of_points(c_4, c_6, p, print_points=False):

    count = 1

    for i in range(p):
        y_squared = ((i ** 3) + c_4 * i + c_6) % p

        for j in range(p):
            if ((j ** 2) % p) == y_squared:
                count += 1

                if print_points:
                    print('(' + str(i) + ', ' + str(j) + ')')
    return count

In [21]:
def weierstrass_order(P, c_4, p):

    sum = P
    order = 1

    while not sum is None:
        sum = weierstrass_addition(sum, P, c_4, p)
        order += 1

    return order

In [22]:
def weierstrass_inverse(P, p):
    if not P is None:
        return P[0], (-1 * P[1]) % p
    else:
        return P

In [23]:
def weierstrass_double(P, c_4, p):

    numerator = (3 * (P[0] ** 2) + c_4) % p
    denominator = (2 * P[1]) % p

    l = (numerator * inverse(denominator, p)) % p

    x = (l ** 2 - 2 * P[0]) % p
    y = (l * (P[0] - x) - P[1]) % p

    return x, y

In [24]:
# P = (x_1, y_1) -> P[0] = x_1, P[1] = y_1
# Q = (x_2, y_2) -> Q[0] = x_2, Q[1] = y_2

def weierstrass_addition_distinct(P, Q, p):

    numerator = (P[1] - Q[1]) % p
    denominator = (P[0] - Q[0]) % p

    l = (numerator * inverse(denominator, p)) % p

    x = (l ** 2 - P[0] - Q[0]) % p
    y = (l * (P[0] - x) - P[1]) % p

    return x, y

In [25]:
# P = (x_1, y_1) -> P[0] = x_1, P[1] = y_1
# Q = (x_2, y_2) -> Q[0] = x_2, Q[1] = y_2

def weierstrass_addition(P, Q, c_4, p):

    if P is None:
        return Q
    if Q is None:
        return P
    if P[0] == Q[0] and P[1] == ((-1 * Q[1]) % p):
        return None
    elif P[0] == Q[0] and P[1] == Q[1]:
        return weierstrass_double(P, c_4, p)
    else:
        return weierstrass_addition_distinct(P, Q, p)

In [26]:
def weierstrass_multiple_additions(P, k, c_4, p):
    Q = None
    for i in range(k):
        Q = weierstrass_addition(Q, P, c_4, p)
    return Q

## Weierstrass curves: solving the discrete logarithm problem using the baby-step giant-step algorithm

In [27]:
def BSGS_print(left_side, right_side, a_0, a_1, m, a, P, P_A, c_4, p):

    for i in range(len(left_side)):
        print(str(i) + 'P = ' + str(left_side[i]))
    print('a_0 = ' + str(a_0))
    print()

    for i in range(len(right_side)):
        print('P_A - ' + str(i) + 'mP = ' + str(right_side[i]))
    print('a_1 = ' + str(a_1))
    print()

    print('m = ' + str(m))
    print('a = ' + str(a))
    Q = None
    for i in range(a):
        Q = weierstrass_addition(Q, P, c_4, p)
        print(str(i + 1) + 'P = ' + str(Q))

    print()
    print('P_A = ' + str(P_A))

In [28]:
def DLP_BSGS(P, P_A, c_4, p, print_intermediate=False):

    N = weierstrass_order(P, c_4, p)
    m = math.floor(math.sqrt(N))

    left_side = [None]
    for i in range(m - 1):
        left_side.append(weierstrass_addition(left_side[i], P, c_4, p))

    right_side = [P_A]
    mP = weierstrass_addition(left_side[-1], P, c_4, p)
    n_mP = weierstrass_inverse(mP, p)

    for i in range(math.floor(N / m) + 1):
        right_side.append(weierstrass_addition(right_side[i], n_mP, c_4, p))

        if right_side[-1] is None:
            a_0 = 0
            a_1 = len(right_side) - 1
            a = (a_0 + m * a_1) % N

            if print_intermediate:
                BSGS_print(left_side, right_side, a_0, a_1, m, a, P, P_A, c_4, p)

            return a

        for j in range(1, len(left_side)):
            a = right_side[-1]
            b = left_side[j]

            if a[0] == b[0] and a[1] == b[1]:
                a_0 = j
                a_1 = len(right_side) - 1
                a = (a_0 + m * a_1) % N

                if print_intermediate:
                    BSGS_print(left_side, right_side, a_0, a_1, m, a, P, P_A, c_4, p)

                return a

In [29]:
def BSGS_print_exam(left_side, right_side, a_0, a_1, m, a, P, P_A, cs, p):

    c_4 = general_to_short(cs, p)[-2]

    print('m = ' + str(m))
    print()

    for i in range(len(left_side)):
        print(str(i) + 'R = ' + str(short_to_general_point(left_side[i], cs, p)))
    print('a_0 = ' + str(a_0))
    print()

    for i in range(len(right_side)):
        print('S - ' + str(i) + 'mR = ' + str(short_to_general_point(right_side[i], cs, p)))
    print('a_1 = ' + str(a_1))
    print()

    print('a = ' + str(a))
    Q = None
    for i in range(a):
        Q = weierstrass_addition(Q, P, c_4, p)
        print(str(i + 1) + 'R = ' + str(short_to_general_point(Q, cs, p)))

    print()
    print('S = ' + str(short_to_general_point(P_A, cs, p)))

In [30]:
def DLP_BSGS_exam(P, P_A, cs, p, print_intermediate=False):

    P = general_to_short_point(P, cs, p)
    P_A = general_to_short_point(P_A, cs, p)
    c_4 = general_to_short(cs, p)[-2]

    N = weierstrass_order(P, c_4, p)
    m = math.floor(math.sqrt(N))

    left_side = [None]
    for i in range(m - 1):
        left_side.append(weierstrass_addition(left_side[i], P, c_4, p))

    right_side = [P_A]
    mP = weierstrass_addition(left_side[-1], P, c_4, p)
    n_mP = weierstrass_inverse(mP, p)

    for i in range(math.floor(N / m) + 1):
        right_side.append(weierstrass_addition(right_side[i], n_mP, c_4, p))

        if right_side[-1] is None:
            a_0 = 0
            a_1 = len(right_side) - 1
            a = (a_0 + m * a_1) % N

            if print_intermediate:
                BSGS_print_exam(left_side, right_side, a_0, a_1, m, a, P, P_A, cs, p)

            return a

        for j in range(1, len(left_side)):
            a = right_side[-1]
            b = left_side[j]

            if a[0] == b[0] and a[1] == b[1]:
                a_0 = j
                a_1 = len(right_side) - 1
                a = (a_0 + m * a_1) % N

                if print_intermediate:
                    BSGS_print_exam(left_side, right_side, a_0, a_1, m, a, P, P_A, cs, p)

                return a

## Weierstrass curves: solving the discrete logarithm problem using Pollard's rho method

In [31]:
def compute_s(W, k, p):
    return (W[0] % p) % k

In [32]:
def update_W(W, P, P_A, s, c_4, p):
    if s == 0:
        return weierstrass_addition(W, P, c_4, p)
    elif s == 1:
        return weierstrass_addition(W, P_A, c_4, p)
    else:
        return weierstrass_addition(W, W, c_4, p)

In [33]:
def update_b(b, s, p):
    if s == 0:
        return (b + 1) % p
    elif s == 1:
        return b % p
    else:
        return (2 * b) % p

In [34]:
def update_c(c, s, p):
    if s == 0:
        return c % p
    elif s == 1:
        return (c + 1) % p
    else:
        return (2 * c) % p

In [35]:
def step(W, b, c, P, P_A, k, c_4, p):
    s = compute_s(W, k, p)
    return update_W(W, P, P_A, s, c_4, p), update_b(b, s, p), update_c(c, s, p)

In [36]:
def DLP_Pollards_rho_print(Ss, S_bs, S_cs, Fs, F_bs, F_cs, a):

    for i in range(len(Ss)):
        print(
            'S' + str(i) + ' = ' + str(Ss[i]) + '\t\t' + 'b' + str(
            i) + ' = ' + str(S_bs[i]) + '\t\t' + 'c' + str(
            i) + ' = ' + str(S_cs[i])
        )

    print()

    for i in range(len(Fs)):
        print(
            'F' + str(i) + ' = ' + str(Fs[i]) + '\t\t' + 'b' + str(
            i) + ' = ' + str(F_bs[i]) + '\t\t' + 'c' + str(
            i) + ' = ' + str(F_cs[i])
        )

    print()
    print('a = ' + str(a))

In [37]:
def DLP_Pollards_rho(P, P_A, S_0, F_0, b_0, c_0, k, c_4, p, print_intermediate=False):

    Ss = [S_0]
    S_bs = [b_0]
    S_cs = [c_0]

    Fs = [F_0]
    F_bs = [b_0]
    F_cs = [c_0]

    while len(Ss) == 1 or not(
        Ss[-1][0] == Fs[-1][0] and Ss[-1][1] == Fs[-1][1]):

        next_S, next_S_b, next_S_c = step(
            Ss[-1], S_bs[-1], S_cs[-1], P, P_A, k, c_4, p
        )
        Ss.append(next_S)
        S_bs.append(next_S_b)
        S_cs.append(next_S_c)

        next_F, next_F_b, next_F_c = step(
            Fs[-1], F_bs[-1], F_cs[-1], P, P_A, k, c_4, p
        )
        next_F, next_F_b, next_F_c = step(
            next_F, next_F_b, next_F_c, P, P_A, k, c_4, p
        )
        Fs.append(next_F)
        F_bs.append(next_F_b)
        F_cs.append(next_F_c)

    a = ((S_bs[-1] - F_bs[-1]) * inverse(S_cs[-1] - F_cs[-1], p)) % p

    if print_intermediate:
        DLP_Pollards_rho_print(Ss, S_bs, S_cs, Fs, F_bs, F_cs, a)

    return a

## Weierstrass curve: solving the discrete logarithm problem using the Pohlig-Hellman attack

In [38]:
def DLP_Pohlig_Hellman(P, Q, c_4, p, N, prime_factorization, print_intermediate=False):

    A = []
    a_is = []

    for i in range(len(prime_factorization)):

        prime_i = prime_factorization[i][0]
        power_i = prime_factorization[i][1]

        Q_i = Q
        A.append([0])
        N_i = int(N / prime_i)
        R_i = weierstrass_multiple_additions(P, N_i, c_4, p)

        if print_intermediate:
            print('i = ' + str(i + 1))
            print('Q_' + str(i + 1) + ' = ' + str(Q_i))
            print('a_' + str(i + 1) + '_-1 = ' + str(A[i][0]))
            print('N_' + str(i + 1) + ' = ' + str(N_i))
            print('R_' + str(i + 1) + ' = ' + str(R_i))
            print()

        for j in range(power_i):

            N_i = int(N / (prime_i ** (j + 1)))

            ap = int(A[i][j] * (prime_i ** (j - 1)))
            apP = weierstrass_multiple_additions(P, ap, c_4, p)
            Q_i = weierstrass_addition(Q_i, weierstrass_inverse(apP, p), c_4, p)

            S_i = weierstrass_multiple_additions(Q_i, N_i, c_4, p)

            A[i].append(DLP_BSGS(R_i, S_i, c_4, p))

            if print_intermediate:
                print('\tj = ' + str(j))
                print('\tN_' + str(i + 1) + ' = ' + str(N_i))
                print('\tQ_' + str(i + 1) + ' = ' + str(Q_i))
                print('\tS_' + str(i + 1) + ' = ' + str(S_i))
                print('\ta_' + str(i + 1) + '_' + str(j) + ' = ' + str(A[i][-1]))
                print()

        a_i = 0
        for j in range(1, len(A[i])):
            a_i += A[i][j] * (prime_i ** (j - 1))
        a_is.append(a_i)

    p_is = []
    for i in range(len(prime_factorization)):
        p_is.append(prime_factorization[i][0] ** prime_factorization[i][1])
        a_is[i] = a_is[i] % p_is[-1]

    if print_intermediate:
        print('a_is = ' + str(a_is))
        print('p_is = ' + str(p_is))

    return CRT(a_is, p_is)

In [39]:
def DLP_Pohlig_Hellman_exam(P, Q, cs, p, N, prime_factorization, print_intermediate=False):

    A = []
    a_is = []

    P = general_to_short_point(P, cs, p)
    Q = general_to_short_point(Q, cs, p)
    c_4 = general_to_short(cs, p)[-2]

    for i in range(len(prime_factorization)):

        prime_i = prime_factorization[i][0]
        power_i = prime_factorization[i][1]

        Q_i = Q
        A.append([0])
        N_i = int(N / prime_i)
        R_i = weierstrass_multiple_additions(P, N_i, c_4, p)

        if print_intermediate:
            print('i = ' + str(i + 1))
            print('p = ' + str(prime_i))
            print('Q_' + str(prime_i) + '_-1 = ' + str(short_to_general_point(Q_i, cs, p)))
            print('a_' + str(prime_i) + '_-1 = ' + str(A[i][0]))
            print('n_' + str(prime_i) + ' = ' + str(N_i))
            print('R_' + str(prime_i) + ' = ' + str(short_to_general_point(R_i, cs, p)))
            print()

        for j in range(power_i):

            N_i = int(N / (prime_i ** (j + 1)))

            ap = int(A[i][j] * (prime_i ** (j - 1)))
            apP = weierstrass_multiple_additions(P, ap, c_4, p)
            Q_i = weierstrass_addition(Q_i, weierstrass_inverse(apP, p), c_4, p)

            S_i = weierstrass_multiple_additions(Q_i, N_i, c_4, p)

            A[i].append(DLP_BSGS(R_i, S_i, c_4, p))

            if print_intermediate:
                print('\tj = ' + str(j))
                print('\tn_' + str(prime_i) + '_' + str(j) + ' = ' + str(N_i))
                print('\tQ_' + str(prime_i) + '_' + str(j) + ' = ' + str(short_to_general_point(Q_i, cs, p)))
                print('\tS_' + str(prime_i) + '_' + str(j) + ' = ' + str(short_to_general_point(S_i, cs, p)))
                print('\ta_' + str(prime_i) + '_' + str(j) + ' = ' + str(A[i][-1]))
                print()

        a_i = 0
        for j in range(1, len(A[i])):
            a_i += A[i][j] * (prime_i ** (j - 1))
        a_is.append(a_i)

    p_is = []
    for i in range(len(prime_factorization)):
        p_is.append(prime_factorization[i][0] ** prime_factorization[i][1])
        a_is[i] = a_is[i] % p_is[-1]

    if print_intermediate:
        print('a_is = ' + str(a_is))
        print('p_is = ' + str(p_is))

    return CRT(a_is, p_is)

## RSA

In [40]:
def RSA_key_generation(p, q, e, print_intermediate=False):

    n = p * q
    phi_n = (p - 1) * (q - 1)
    d = inverse(e, phi_n)

    if print_intermediate:
        print('p = ' + str(p))
        print('q = ' + str(q))
        print('e = ' + str(e))
        print('n = ' + str(n))
        print('phi_n = ' + str(phi_n))
        print('d = ' + str(d))

    return d

In [41]:
def RSA_encryption(m, e, n):
    return pow(m, e, n)

In [42]:
def RSA_decryption(c, d, n):
    return pow(c, d, n)

## RSA-CRT

In [43]:
def RSA_CRT_key_generation(p, q, e, print_intermediate=False):

    n = p * q
    phi_n = (p - 1) * (q - 1)
    d = inverse(e, phi_n)
    d_p = d % (p - 1)
    d_q = d % (q - 1)
    u = inverse(p, q)

    if print_intermediate:
        print('p = ' + str(p))
        print('q = ' + str(q))
        print('e = ' + str(e))
        print('n = ' + str(n))
        print('phi_n = ' + str(phi_n))
        print('d = ' + str(d))
        print('d_p = ' + str(d_p))
        print('d_q = ' + str(d_q))
        print('u = ' + str(u))

    return n, p, q, d_p, d_q, u

In [44]:
def RSA_CRT_encryption(m, e, n):
    return RSA_encryption(m, e, n)

In [45]:
def RSA_CRT_decryption(c, p, q, d_p, d_q, u, print_intermediate=False):

    c_p = c % p
    c_q = c % q
    m_p = pow(c_p, d_p, p)
    m_q = pow(c_q, d_q, q)
    m = (m_p + p * u * (m_q - m_p)) % (p * q)

    if print_intermediate:
        print('c_p = ' + str(c_p))
        print('c_q = ' + str(c_q))
        print('m_p = ' + str(m_p))
        print('m_q = ' + str(m_q))
        print('m = ' + str(m))

    return m

## Factorization using the p-1 method

In [46]:
def lcm(A):
    lcm = 1
    for x in A:
        lcm = math.lcm(lcm, x)
    return lcm

In [47]:
def p_1_method(n, a, s, print_intermediate=False):

    s = lcm(s)
    if print_intermediate:
        print('s = ' + str(s))
        print()

    b = (a ** s) % n
    d = math.gcd(b - 1, n)

    if print_intermediate:
        print('b = ' + str(b))
        print('d = ' + str(d))
        print()

    if not d == 1 and not d == n:
        return d, int(n / d)
    else:
        return None

## Factorization using the Pollard's rho with Floyd's cycle-finding method

In [48]:
def g(x, c, n):
    return (x ** 2 + c) % n

In [49]:
def factorize_Pollards_rho(x, c, n, print_intermediate=False):

    slow_walk = x
    fast_walk = x
    d = 1
    S = 1

    if print_intermediate:
        print('Slow walk = ' + str(slow_walk) + ',\t\tFast walk = ' + str(fast_walk) + ',\t\td = ' + str(d))

    while d == 1:
        slow_walk = g(slow_walk, c, n)
        fast_walk = g(g(fast_walk, c, n), c, n)
        S *= abs(slow_walk - fast_walk)
        d = math.gcd(abs(slow_walk - fast_walk), n)

        if print_intermediate:
            print('Slow walk = ' + str(slow_walk) + ',\t\tFast walk = ' + str(fast_walk) + ',\t\td = ' + str(d))

    if print_intermediate:
        print('S = ' + str(S))
        print('gcd(S, n) = ' + str(math.gcd(S, n)))

    if not d == n:
        return d
    else:
        return None

## The discrete logarithm problem for numbers

In [50]:
def BSGS_print_numbers(left_side, right_side, i, j, n, m, a):

    print('n = ' + str(n))
    print('m = ' + str(m))
    print()

    for k in range(len(left_side)):
        print('g^' + str(k) + ' = ' + str(left_side[k]))
    print('j = ' + str(j))
    print()

    print('c = g^(-m)')
    for l in range(len(right_side)):
        print('h * c^' + str(l) + ' = ' + str(right_side[l]))
    print('i = ' + str(i))
    print()

    print('a = (j + i * m) % n = ' + str(a))

In [51]:
def DLP_BSGS_numbers(g, h, p, n, print_intermediate=False):

    m = math.ceil(math.sqrt(n))

    left_side = [1]
    for i in range(m - 1):
        left_side.append((left_side[i] * g) % p)

    g_nm = inverse(g ** m, p)
    right_side = [h]
    for i in range(m - 1):
        a = right_side[i]

        for j in range(len(left_side)):
            b = left_side[j]

            if a == b:

                logarithm = (j + i * m) % n

                if print_intermediate:
                    BSGS_print_numbers(left_side, right_side, i, j, n, m, logarithm)

                return logarithm

        right_side.append((right_side[i] * g_nm) % p)

In [52]:
def DLP_Pohlig_Hellman_numbers(g, h, p, n, prime_factorization, print_intermediate=False):

    A = []
    a_is = []

    for i in range(len(prime_factorization)):

        prime_i = prime_factorization[i][0]
        power_i = prime_factorization[i][1]

        h_i = h
        A.append([0])
        n_i = int(n / prime_i)
        r_i = pow(g, n_i, p)

        if print_intermediate:
            print('i = ' + str(i + 1))
            print('prime = ' + str(prime_i))
            print('H_' + str(prime_i) + '_-1 = ' + str(h_i))
            print('b_' + str(prime_i) + '_-1 = ' + str(A[i][0]))
            print('p_' + str(prime_i) + ' = ' + str(n_i))
            print('g_' + str(prime_i) + ' = ' + str(r_i))
            print()

        for j in range(power_i):

            n_i = int(n / (prime_i ** (j + 1)))

            ap = int(A[i][j] * (prime_i ** (j - 1)))
            g_ap = pow(g, ap, p)
            h_i = (h_i * pow(g_ap, -1, p)) % p

            s_i = pow(h_i, n_i, p)

            A[i].append(DLP_BSGS_numbers(r_i, s_i, p, pow(prime_i, 2)))

            if print_intermediate:
                print('\tj = ' + str(j))
                print('\tp_' + str(prime_i) + '_' + str(j) + ' = ' + str(n_i))
                print('\tH_' + str(prime_i) + '_' + str(j) + ' = ' + str(h_i))
                print('\th_' + str(prime_i) + '_' + str(j) + ' = ' + str(s_i))
                print('\tb_' + str(prime_i) + '_' + str(j) + ' = ' + str(A[i][-1]))
                print()

        a_i = 0
        for j in range(1, len(A[i])):
            a_i += A[i][j] * (prime_i ** (j - 1))
        a_is.append(a_i)

    p_is = []
    for i in range(len(prime_factorization)):
        p_is.append(prime_factorization[i][0] ** prime_factorization[i][1])
        a_is[i] = a_is[i] % p_is[-1]

    if print_intermediate:
        print('a_is = ' + str(a_is))
        print('p_is = ' + str(p_is))

    return CRT(a_is, p_is)

In [53]:
def DLP_Pollards_rho_GH(G, H, p, n, rs, ts, r_0, t_0, print_intermediate=False):

    s = []
    for i in range(len(rs)):
        s.append((pow(G, rs[i], p) * pow(H, ts[i], p)) % p)

    x_0 = (pow(G, r_0, p) * pow(H, t_0, p)) % p

    slow = [x_0]
    s_powers = [(r_0, t_0)]

    fast = [x_0]
    f_powers = [(r_0, t_0)]
    no_collision = True

    while no_collision:
        i = slow[-1] % len(s)
        slow.append((slow[-1] * s[i]) % p)
        s_powers.append(((s_powers[-1][0] + rs[i]) % n, (s_powers[-1][1] + ts[i]) % n))

        j = fast[-1] % len(s)
        fast_ = (fast[-1] * s[j]) % p
        f_powers_ = ((f_powers[-1][0] + rs[j]) % n, (f_powers[-1][1] + ts[j]) % n)

        k = fast_ % len(s)
        fast.append((fast_ * s[k]) % p)
        f_powers.append(((f_powers_[0] + rs[k]) % n, (f_powers_[1] + ts[k]) % n))

        if slow[-1] == fast[-1]:
            no_collision = False

    if print_intermediate:
        print('s: ' + str(s))
        print("Slow walk's powers: " + str(s_powers))
        print('Slow walk: ' + str(slow))
        print("Fast walk's powers: " + str(f_powers))
        print('Fast walk: ' + str(fast))

    return ((s_powers[-1][0] - f_powers[-1][0]) * pow(f_powers[-1][1] - s_powers[-1][1], -1, n)) % n